# Playground
Figuring out how to work with the NEMO model.

## Objectives
1. Simulate each action (project, associate, merge) several times.
2. For a given brain region $A$ (where we simulated activity) output a matrix $W \in \mathbb{R}^{n \times t}$, where $n$ is number of neurons, $t$ is number of rounds, and $W_{ij}$ is the synaptic weight of neuron $i$ and time step $j$. We will cal this the **neural activity matrix** of $A$.
    - Optional: Try out various combinations of brain regions and actions (ie., have multiple brain regions that project onto a single region where you extract weights from).
    - Optional: Try the above for complex NEMO models, like sentence parsing or text generation.
3. Run nonlinear (and maybe linear?) dimension reductions to achieve a neural manifold.
    - Time Permits: compare this artifical neural manifold to a real neural manifold (though utility of comparison depends on tasks that these manifolds were dervied from).

## NEMO Library
The following files will be useful for our experiments:
- `simulations.py`: contains code for running simulations of different NEMO actions.
- `brain.py`: contains (all?) code for the NEMO model. We may need to modify this for our use case.
- `brain_util.py`: contains code for saving/loading NEMO instances.

In [7]:
import nemo.brain as brain
import numpy as np

# Understanding and Modifying NEMO Library
Let us look at an example of simulating the project operation (which comes from `simulations.py`).

In [ ]:
def project_sim(n=1_000_000, k=1_000, p=0.01, beta=0.05, t=50):
    """Simulate repeated projections from a stimulus to a brain area.
    
    Args:
        n: Number of neurons in an area A
        k: Number of winners in A that fire at each round (instantaneous assembly size)
        p: Connection probability
        beta: Plasticity parameter
        t: Number of projection rounds
    
    Returns:
        List of support sizes over time: at each round, the number of distinct
        neurons in A that have ever fired at least once.
    """
    b = brain.Brain(p)
    
    # Create a stimulus (called "stim") modeled as k neurons that will fire
    # together whenever "stim" is present.
    b.add_stimulus("stim", k)
    
    # Add a brain region (named "A") with n neurons. On each projection step,
    # exactly k of these n neurons are chosen as winners (the active assembly
    # in A at that time step).
    b.add_area("A", n, k, beta)
        
    # First step: let the stimulus "stim" project into A. The k most excited
    # neurons in A fire, forming an initial assembly of size k tied to "stim".
    b.project({"stim": ["A"]}, {})
    
    for i in range(t - 1):
        # Subsequent steps: both the stimulus and the current winners in A
        # project into A. This combines feedforward input from "stim" with
        # recurrent input from A itself, further strengthening and growing
        # the assembly over time.
        b.project({"stim": ["A"]}, {"A": ["A"]})
    
    return b.area_by_name['A'].saved_w

no_neurons_fired_time = project_sim()
assert len(no_neurons_fired_time) == 50

Some things to note:
- The number of neurons in region $A$ is denoted by `n`. In the above, `n = 1_000_000`. The number of rounds we are projecting for is `t = 50`. This produces a very large neural activity matrix $W \in \mathbb{R}^{1,000,000 \times 50}$. **We should be careful and think about memory**.
- Instead of the above, we could only consider the neurons associated with the assembly, denoted by `k`. In the above, `k = 1_000` wich means we would produce a neural activity matrix $W \in \mathbb{R}^{1,000 \times 50}$.
- The implementation above is a laxy implemenation (we never truly instantiate `n` neurons). To be able to keep track of the synaptic weights, we would need to use `b.add_explicit_area(...)`.

Recall that we do two kind of projections: (i) stimulus into $A$; (ii) $A$ into $A$. There are two weights we can then extract:
```py
b.connectomes_by_stimulus["stim"]["A"] # Length n array. Synaptic weights from stimulus into area A.
b.connectomes["A"]["A"]                # Matrix of size (n, n). Synaptic weight from one neuron in area A to another in area A.
```

Below is some code from GPT offering multiple solutions.

In [ ]:
def project_sim_with_weight_logging(
    n                   = 10_000,
    k                   = 1_000,
    p                   = 0.01,
    beta                = 0.05,
    t                   = 50,
    record_mode         = "feedforward", # "feedforward", "recurrent", or "both"
    recurrent_aggregate = "incoming",    # "incoming", "outgoing", "total", or "self"
):
    """
    Run T projection rounds and record synaptic weights for all n neurons in area A.

    Args:
        n: Number of neurons in an area A
        k: Number of winners in A that fire at each round (instantaneous assembly size)
        p: Connection probability
        beta: Plasticity parameter
        t: Number of projection rounds
        
        record_mode:
            "feedforward": record only stim -> A weights.
            "recurrent"  : record only A -> A recurrent weights (summarized per neuron).
            "both"       : record both, in parallel.
        
        recurrent_aggregate:
            How to reduce the n x n recurrent matrix to a length-n vector:
                "incoming": sum over presynaptic neurons (column sums), which is total recurrent input into each neuron.
                "outgoing": sum over postsynaptic neurons (row sums), which is total recurrent output from each neuron.
                "total"   : incoming + outgoing.
                "self"    : diagonal (self-connections only).

    Returns:
        A dictionary with:
            result["saved_w"]    : list of length t - support size (ever-fired neurons in A) after each round.
            result["feedforward"]: 2D array of shape (n, t) or None.
                                   Assume you save the matrix as recurrent Then, feedforward[i, step] = weight
                                   from "stim" -> neuron i in A after that step.
            result["recurrent"]  : 2D array of shape (n, t) or None.
                                   Assume you save the matrix as recurrent. Then, recurrent[i, step] = aggregated
                                   recurrent weight for neuron i in A after that step (definition controlled by
                                   recurrent_aggregate).
    """
    assert record_mode in ("feedforward", "recurrent", "both"), ("Invalid record_mode")
    assert recurrent_aggregate in ("incoming", "outgoing", "total", "self"), ("Invalid recurrent_aggregate")

    b = brain.Brain(p)
    b.add_stimulus("stim", k)

    # Explicit area: all n neurons and their synapses are materialized.
    b.add_explicit_area("A", n, k, beta)

    feedforward_history = []  # list of (n,) arrays
    recurrent_history   = []  # list of (n,) arrays

    def snapshot():
        """Record weights after a projection step."""
        if record_mode in ("feedforward", "both"):
            # Vector of weights from stim -> each neuron in A (shape (n,))
            w_ff = b.connectomes_by_stimulus["stim"]["A"].copy()
            feedforward_history.append(w_ff)

        if record_mode in ("recurrent", "both"):
            # Full recurrent matrix A -> A (shape (n, n))
            W_rec = b.connectomes["A"]["A"]
            
            # The recorded synaptic weights (initialze)
            rec_vec = np.zeros(n)

            if recurrent_aggregate == "incoming":
                # Total recurrent input into each neuron (sum over presynaptic j)
                # column i = sum_j W_rec[j, i]
                rec_vec = W_rec.sum(axis=0)

            elif recurrent_aggregate == "outgoing":
                # Total recurrent output from each neuron (sum over postsynaptic i)
                # row j = sum_i W_rec[j, i]
                rec_vec = W_rec.sum(axis=1)

            elif recurrent_aggregate == "total":
                # Incoming + outgoing for each neuron
                rec_in = W_rec.sum(axis=0)
                rec_out = W_rec.sum(axis=1)
                rec_vec = rec_in + rec_out

            elif recurrent_aggregate == "self":
                # Only self-connections W_rec[i, i]
                rec_vec = np.diag(W_rec)

            recurrent_history.append(rec_vec.copy())

    # ============================================== PROJECTION STEPS ==============================================
    # Step 1: only stim -> A (no recurrent yet)
    b.project({"stim": ["A"]}, {})
    snapshot()

    # Steps 2, ..., t: stim -> A and A -> A (recurrent)
    for _ in range(t - 1):
        b.project({"stim": ["A"]}, {"A": ["A"]})
        snapshot()

    # =============================================== SAVING RESULTS ================================================
    result = {
        "saved_w": b.area_by_name["A"].saved_w,
        "feedforward": None,
        "recurrent": None,
    }

    if feedforward_history:
        # Shape (n, t): row = neruon index | col = time step
        result["feedforward"] = np.stack(feedforward_history, axis=0).T

    if recurrent_history:
        # Shape (n, t): row = neruon index | col = time step
        result["recurrent"] = np.stack(recurrent_history, axis=0).T

    return result

In [ ]:
# 1) Only feedforward weights: W_ff[step, i] = stim -> A_i
res_ff = project_sim_with_weight_logging(
    n=2000, k=200, p=0.01, beta=0.05, t=20,
    record_mode="feedforward"
)
W_ff = res_ff["feedforward"] # shape (2000, 20)

# 2) Only recurrent incoming weights: W_rec[step, i] = sum_j W_rec[j,i]
res_rec_in = project_sim_with_weight_logging(
    n=2000, k=200, p=0.01, beta=0.05, t=20,
    record_mode="recurrent",
    recurrent_aggregate="incoming"
)
W_rec_in = res_rec_in["recurrent"] # shape (2000, 20)

# 3) Both feedforward and recurrent (e.g., outgoing)
res_both = project_sim_with_weight_logging(
    n=2000, k=200, p=0.01, beta=0.05, t=20,
    record_mode="both",
    recurrent_aggregate="outgoing"
)
W_ff      = res_both["feedforward"] # shape (2000, 20)
W_rec_out = res_both["recurrent"]   # shape (2000, 20)
